![alt text](https://drive.google.com/uc?export=view&id=1UXScsVx_Wni_JuDdB8LeTnM6jsPfIwkW)

## Inference for Pneumonia Detection - EfficientDet

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


References 
1. Original paper: https://arxiv.org/pdf/1911.09070.pdf
2. Repository clone: !git clone https://github.com/kamauz/EfficientDet.git


#### import libraries

# **Inference**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
import json
import numpy as np
import os
import time
import glob
# Input data files are available in the
dataDir = '/content/drive/MyDrive/Capstone/'
testDataDir = dataDir+'test_gray/'

In [ ]:
%cd /content/drive/MyDrive/Capstone/black/EfficientDet/

/content/drive/MyDrive/Capstone/black/EfficientDet


In [ ]:
!git clone https://github.com/kamauz/EfficientDet.git

fatal: destination path 'EfficientDet' already exists and is not an empty directory.


In [ ]:
%cd /content/drive/MyDrive/Capstone/black/EfficientDet/EfficientDet/

/content/drive/MyDrive/Capstone/black/EfficientDet/EfficientDet


In [ ]:
# forked a github repository to add fine tuning to efficient det
!python setup.py build_ext --inplace

running build_ext


In [ ]:
# installing require packages
!pip install -r requirements.txt
!pip install pycocotools>=2.0.2
!pip install effdet
!pip install effdet --upgrade
!pip install focal


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-zr439rdj
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-zr439rdj
     |████████████████████████████████| 337kB 5.6MB/s 
     |████████████████████████████████| 30.6MB 104kB/s 
     |████████████████████████████████| 25.0MB 132kB/s 
     |████████████████████████████████| 2.1MB 48.0MB/s 
     |████████████████████████████████| 411.5MB 40kB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 3.8MB 47.8MB/s 
     |████████████████████████████████| 512kB 45.8MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263909 sha256=e0f63fbcd502504a17d13af85f664c5da09c0df12f799e6f80b8196a7e7c3bfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-jww5m32u/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
  Created wheel for

     |████████████████████████████████| 112kB 4.2MB/s 
     |████████████████████████████████| 245kB 25.6MB/s 
     |████████████████████████████████| 645kB 50.1MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=263990 sha256=8f6b7f5154ae3edc3bf90a09e6780c3911a8dd38b507fb21f6288e98c37d8cb7
  Stored in directory: /root/.cache/pip/wheels/68/a5/e7/56401832f23d0b2db351c5b682e466cb4841960b086da65e4e
Successfully built pycocotools
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


Requirement already up-to-date: effdet in /usr/local/lib/python3.7/dist-packages (0.2.1)
     |████████████████████████████████| 51kB 3.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 


In [ ]:
def preprocess_image(image, image_size):
    # image, RGB
    image_height, image_width = image.shape[:2]
    if image_height > image_width:
        scale = image_size / image_height
        resized_height = image_size
        resized_width = int(image_width * scale)
    else:
        scale = image_size / image_width
        resized_height = int(image_height * scale)
        resized_width = image_size

    image = cv2.resize(image, (resized_width, resized_height))
    image = image.astype(np.float32)
    image /= 255.
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image -= mean
    image /= std
    pad_h = image_size - resized_height
    pad_w = image_size - resized_width
    image = np.pad(image, [(0, pad_h), (0, pad_w), (0, 0)], mode='constant')

    return image, scale

def postprocess_boxes(boxes, scale, height, width):
    c_boxes = boxes.copy()
    c_boxes /= scale
    c_boxes[:, 0] = np.clip(c_boxes[:, 0], 0, width - 1)
    c_boxes[:, 1] = np.clip(c_boxes[:, 1], 0, height - 1)
    c_boxes[:, 2] = np.clip(c_boxes[:, 2], 0, width - 1)
    c_boxes[:, 3] = np.clip(c_boxes[:, 3], 0, height - 1)
    return c_boxes

def draw_boxes(image, boxes, scores, labels, colors, classes):
    for b, l, s in zip(boxes, labels, scores):
        class_id = int(l)
        class_name = classes[class_id]
    
        xmin, ymin, xmax, ymax = list(map(int, b))
        score = '{:.4f}'.format(s)
        color = colors[class_id]
        label = '-'.join([class_name, score])
    
        ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 1)
        cv2.rectangle(image, (xmin, ymax - ret[1] - baseline), (xmin + ret[0], ymax), color, -1)
        cv2.putText(image, label, (xmin, ymax - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

In [ ]:
from model import efficientdet
from utils import preprocess_image, postprocess_boxes
from utils.draw_boxes import draw_boxes

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
phi = 4
weighted_bifpn = True
model_path = '/content/drive/MyDrive/Capstone/black/EfficientDet/EfficientDet/weights.effidet.h5'
image_sizes = (512, 640, 768, 896, 1024, 1280, 1408)
image_size = image_sizes[phi]
# Pnuemonia classes
classes = {value['id'] - 1: value['name'] for value in json.load(open('coco_1.json', 'r')).values()}
num_classes = 1
score_threshold = 0.3
colors = [np.random.randint(0, 256, 3).tolist() for _ in range(num_classes)]
prediction_model, model = efficientdet(phi=phi,
                            weighted_bifpn=weighted_bifpn,
                            num_classes=num_classes,
                            score_threshold=score_threshold,
                            finetuned_num_classes=1)
model.load_weights(model_path, by_name=True)
model.summary()

Model: "efficientdet_p"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 512, 512, 48) 1296        input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 512, 512, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 512, 512, 48) 0           stem_bn[0][0]                    
_____________________________________________________________________________________

In [ ]:
for image_path in glob.glob('/content/drive/MyDrive/Capstone/10/*.jpg'): # 000e3a7d-c0ca-4349-bb26-5af2d8993c3d
    image = cv2.imread(image_path)
    src_image = image.copy()
    # BGR -> RGB
    image = image[:, :, ::-1]
    h, w = image.shape[:2]

    image, scale = preprocess_image(image, image_size=image_size)
    # run network
    start = time.time()
    boxes, scores, labels = model.predict_on_batch([np.expand_dims(image, axis=0)])
    boxes, scores, labels = np.squeeze(boxes), np.squeeze(scores), np.squeeze(labels)
    print(time.time() - start)
    boxes = postprocess_boxes(boxes=boxes, scale=scale, height=h, width=w)

    # select indices which have a score above the threshold
    indices = np.where(scores[:] > score_threshold)[0]

    # select those detections
    boxes = boxes[indices]
    labels = labels[indices]

    draw_boxes(src_image, boxes, scores, labels, colors, classes)

    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow('image', src_image)
    cv2.waitKey(0)

StagingError: ignored